# TD 3

### **Exercice 1 : Utilisation des modules os et os.path**
1. Commencer par regarder la documentation. Pour tester ces exercices, créer un répertoire contenant une arborescence de quelques sous-répertoires, fichiers (vides), et liens symboliques.

2. Écrire un programme qui fait la liste des fichiers (pas les répertoires!) présents dans un répertoire donné (répertoire et sous-répertoires). (Et qui affiche la liste, avec un nom par ligne ; exactement comme la commande find .). S'il fonctionne sur le répertoire de test, l'essayer sur votre répertoire racine.

In [3]:
import os

def find(root):
    tree = os.walk(root)
    for root, dirs, files in tree:
        for file in files:
            # print(os.path.join(root, file))
            print(root+os.path.sep+file)

find('folder')

folder\.hidden
folder\touch
folder\another_folder\file.txt
folder\another_folder\next_folder\.hidden_file
folder\another_folder\next_folder\a_file
folder\another_folder\next_folder\but_not_least\i_like_pizza
folder\another_folder\next_folder\last_folder\hello


3. Indiquer le nombre de fichiers et le nombre de répertoires. (Comme si on exécutait la commande find . -type f | wc -l ; find . -type d | wc -l)

In [5]:
import os
def find_count(root):
    nfile = ndir = 0
    for root, dirs, files in os.walk(root):
        nfile += len(files)
        ndir += len(dirs)
    print("number of files : %d" % nfile)
    print("number of directories : %d" % ndir)
    
find_count('folder')

number of files : 7
number of directories : 5


4. Indiquer le nombre de fichiers, le nombre de répertoires, et le nombre de liens symboliques. (Si il y a un lien toto qui pointe vers le répertoire truc, alors toto compte uniquement comme un lien symbolique -- pas comme un répertoire ; et il ne faut pas descendre dans ses sous-répertoires.)

In [6]:
def find_count2(root):
    nfile = ndir = nsym = 0
    for root, dirs, files in os.walk(root):
        for directory in dirs:
            if os.path.islink(root+os.path.sep+directory):
                nsym += 1
            else:
                ndir += 1
        for file in files:
            if os.path.islink(os.path.join(root, file)):
                nsym += 1
            else:
                nfile += 1
    print("number of files : " + str(nfile))
    print("number of directories : " + str(ndir))
    print("number of symlinks : " + str(nsym))
    
find_count2("folder")

number of files : 7
number of directories : 4
number of symlinks : 1


5. Indiquer les mêmes nombres, mais en différenciant les éléments « publics » et les éléments « privés ». Un fichier est considéré comme « privé » si le groupe propriétaire et les autres utilisateurs n'ont aucun droit dessus ; sinon il est public.

In [7]:
def count_stats(root):
    nprivfile = npubfile = nprivdir = npubdir = nprivsym = npubsym = 0
    for root, dirs, files in os.walk(root):
        for d in dirs:
            try:
                path = os.path.join(root, d)
                is_public = os.stat(path).st_mode%64 # public if it is not 0
                if os.path.islink(path):
                    if is_public: npubsym += 1
                    else: nprivsym += 1
                else:
                    if is_public: npubdir += 1
                    else: nprivdir += 1
            except Exception as e: print(e)
        for f in files:
            try:
                path = os.path.join(root, f)
                is_public = os.stat(path).st_mode%64 # public if it is not 0
                if os.path.islink(path):
                    if is_public: npubsym += 1
                    else: nprivsym += 1
                else:
                    if is_public: npubfile += 1
                    else: nprivfile += 1
            except Exception as e: print(e)
    print("%s :"%root)
    print("Files: \t\t[public: %d]\t[private: %d]" % (npubfile, nprivfile))
    print("Dirs: \t\t[public: %d]\t[private: %d]" % (npubdir, nprivdir))
    print("Symlinks:\t[public: %d]\t[private: %d]" % (npubsym, nprivsym))
                    
count_stats('folder')
count_stats('.')

folder\another_folder\next_folder\last_folder :
Files: 		[public: 7]	[private: 0]
Dirs: 		[public: 4]	[private: 0]
Symlinks:	[public: 1]	[private: 0]
.\folder\another_folder\next_folder\last_folder :
Files: 		[public: 9]	[private: 0]
Dirs: 		[public: 6]	[private: 0]
Symlinks:	[public: 1]	[private: 0]


### **Exercice 2 (modules os, sys, hashlib)**
Il arrive souvent qu'un même fichier se retrouve présent avec des noms différents à divers emplacements. On peut, par exemple, avoir téléchargé plusieurs fois un même mp3 ou un document pdf, depuis des sites différents qui l'auront nommé différemment, et placé dans différents sous-répertoires.

Il peut être coûteux de comparer deux à deux de gros fichiers binaires, mais une fonction de hachage comme md5 ou sha1 permet de détecter les fichiers identiques avec une quasi-certitude (module hashlib). On demande d'écrire un script duplicates qui prend comme paramètres optionnels une extension de nom de fichier ( -e, --extension, defaut : None) un répertoire ( -d, --directory, défaut : répertoire courant) et un fichier de sortie (-o, --output, défaut : sortie standard), et renvoie les chemins complets vers les groupes de fichiers identiques, une ligne par fichier, et une ligne vide entre chaque groupe.

Par exemple :

```
jyt@drobie:~/monge> duplicates -e pdf -d tmp

tmp/f38w5635114j1957.pdf
tmp/qdesar.pdf

tmp/brunet.pdf
tmp/PhysRevE_68_041101.pdf

tmp/eis.pdf
tmp/JEDP_1999____A15_0.pdf

tmp/exos07.pdf
tmp/exos07-1.pdf
tmp/exos07-2.pdf

tmp/kshg21g4.pdf
tmp/be9kd6kh.pdf
tmp/uq8uiawn.pdf
```

In [2]:
#! /usr/local/bin/python3
'''duplicate.py: Program that searches for duplicate files.

Usage: duplicate.py [-h] [--help] [-e] [--extension] [extension] [-d] [--directory] [root] [-o] [--output] [output_file]
'''

import os, sys, getopt, hashlib

def usage():
	'Print usage doc and exit.'
	print (__doc__)
	sys.exit()


if __name__ == '__main__':

	try:
		optlist, arglist = getopt.getopt(sys.argv[1:], 'he:d:o:', ['help', '--extension=', '--directory=', 'output='])
	except:
		usage()

	if not optlist:
		usage()

	ext = root = ""
	for opt, arg in optlist:
		if opt in ['-h', '--help']: usage()
		elif opt in ['-e', '--extension']: ext = arg
		elif opt in ['-d', '--directory']: root = arg
		elif opt in ['-o', '--output']: output = arg
	
	duplicates = {}

	for root, dirs, files in os.walk(root):
		for file in files:
			path = os.path.join(root, file)
			try:
				with open(path, 'rb') as f:
					md5 = hashlib.md5(f.read()).hexdigest()
					if md5 in duplicates:
						duplicates[md5].append(path)
					else:
						duplicates[md5] = [path]
			except Exception as e: print(e)

	for md5, path_list in duplicates.items():
		if len(path_list) > 1:
			print()
			for path in path_list:
				print(path)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



duplicate.py: Program that searches for duplicate files.

Usage: duplicate.py [-h] [--help] [-e] [--extension] [extension] [-d] [--directory] [root] [-o] [--output] [output_file]

Traceback (most recent call last):
  File "<ipython-input-2-7b4c2d04148a>", line 18, in <module>
    optlist, arglist = getopt.getopt(sys.argv[1:], 'he:d:o:', ['help', '--extension=', '--directory=', 'output='])
  File "E:\Programmes\Anaconda\lib\getopt.py", line 95, in getopt
    opts, args = do_shorts(opts, args[0][1:], shortopts, args[1:])
  File "E:\Programmes\Anaconda\lib\getopt.py", line 195, in do_shorts
    if short_has_arg(opt, shortopts):
  File "E:\Programmes\Anaconda\lib\getopt.py", line 211, in short_has_arg
    raise GetoptError(_('option -%s not recognized') % opt, opt)
getopt.GetoptError: option -f not recognized

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "E:\Programmes\Anaconda\lib\site-packages\IPython\core\interactiveshell

SystemExit: 

E:\Programmes\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


### **Exercice 3 (module re)**
La première étape de la construction d'un correcteur orthographique consiste à écrire une fonction qui vérifie si un mot est dans le dictionnaire (cf. TD1). La seconde consiste à proposer des corrections pour les mots qui n'y sont pas.

Deux mots u et v sont dits à distance d'édition 1 s'ils diffèrent par remplacement, insertion ou suppression d'un caractère.

Ecrire une fonction lookup(w) qui prend comme argument un mot arbitraire w et retourne la liste des mots du dictionnaire qui sont à distance 1 (ou 0) de w. On utilisera une expression régulière judicieusement construite. On pourra reprendre le dictionnaire du TD1.

Par exemple, on devrait avoir

```
>>> for x in lookup('ais'): print x,
... 
agis ai aie aies ail ails air airs ais aïs aise aisé aisy ait amis anis ans
apis ars as avis axis bais bis dais dis fais fis gais gis hais jais lais lis
mais mis nais ois pais pis rais ris sais sis tais vais vis
>>> 
```

In [3]:
def lookup(word):
    with open('liste.de.mots.francais.frgut.txt', 'r') as fr_dict:
        candidates = [x for x in fr_dict.readlines() if len(x) >= len(word) - 1 && len(x) <= len(word)+1]
    return candidates

print(lookup('hello'))

SyntaxError: invalid syntax (<ipython-input-3-3ea74646d1e5>, line 3)